#  TP 2: 

### 18 mars 2024

### Par Samuel Fortin, Philippe Truchon et Benjamin Trudel

## TP2.1 Décomposition QR par la méthode Householder

#### Fonctions générales

In [350]:
import fnmatch
import functools
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math


sns.set_theme(style="ticks", palette="deep")

plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False



# Liste les noms de fichier d'un dossier
def listNameOfFiles(directory: str, extension="csv"):
    found_files = []
    for file in os.listdir(directory):
        if fnmatch.fnmatch(file, f"*.{extension}"):
            found_files.append(file)
    return found_files


# Lis et crée une matrice numpy à partir de chemin d'un fichier texte
def readTXT(path: str, header):
    fich = open(path, "r")
    fich_str = list(fich)
    fich.close()
    x = []
    for i in fich_str[header:]:
        elem_str = i.replace("\n", "")
        x.append(float(elem_str))
    return np.array(x)

def readcsv(path: str, header):
    arr = np.loadtxt(path,
                 delimiter=",", dtype=float, skiprows=header)
    return arr


path = os.path.abspath("")
files_name = listNameOfFiles(path)


### Questions:

### a)

<i>À l’aide des équations (2.1.2) et (2.1.3), démontrez que les matrices de réflexion Q_i sont orthogonales.</i>

Sachant que $v^T v = I$:

\begin{gather*}
 H_{m,i}H_{m,i}^T = H_{m,i}H_{m,i}
 \\
 H_{m,i}H_{m,i}^T = \Big( I - \frac{2 v v^T}{v^T v} \Big) \Big( I - \frac{2 v v^T}{v^T v} \Big)
 \\
 H_{m,i}H_{m,i}^T = I - \frac{ 4 v v^T }{ v^T v } + \frac{ 4 v (v^T v) v^T }{ (v^T v)^2 }
 \\
 H_{m,i}H_{m,i}^T = I - \frac{ 4 v v^T }{ v^T v } + \frac{ 4 v v^T }{ v^T v }
 \\
 H_{m,i}H_{m,i}^T = I
\end{gather*}  


$$ Q_i = \left [
\begin{matrix}
I_i & 0 \\
0 & H_{m,i} 
\end{matrix}
\right ]$$

En utilisant la forme générale du produit des matrices par blocs, on sait que:

$$ M^T = \left [
\begin{matrix}
A^T & B^T \\
C^T & D^T 
\end{matrix}
\right ] $$

Donc:

$$ Q_i = \left [
\begin{matrix}
I_i & 0 \\
0 & H_{m,i} 
\end{matrix}
\right ] = \left [
\begin{matrix}
I_i^T & 0 \\
0 & H_{m,i}^T 
\end{matrix}
\right ] = Q_i^T$$





### b)

<i>Démontrez l’équation (2.1.5) et que la matrice Q est orthogonale.</i>

 

En utilisant la propriété d'associativité : $(AB)^T = B^T A^T$ et sachant que $Q_{i}^T Q_{i} = I$

\begin{gather*}
    Q = \prod_{i=0}^{n-1} Q_i^T
\end{gather*}

\begin{gather*}
    (\prod_{i=0}^{n-1} Q_i)^T (\prod_{i=0}^{n-1} Q_i) = (\prod_{i=n-1}^{0} Q_i^T)(\prod_{i=0}^{n-1} Q_i) = \prod_{i=0}^{n-1} I = I
\end{gather*}

La matrice Q est donc orthogonale.

### c)

<i>Implémentez la fonction householder_qr qui prend en argument une matrice A et qui retourne les matrices Q
et R obtenues par la méthode de Householder.</i>


In [351]:
def householder_qr(A, inter=False):
    m, n = A.shape
    Q = np.identity(m)
    for i in range(n):
        H = np.identity(m)
        x = A[i:, i]

        e1 = np.zeros(len(x)).T
        e1[0] = 1

        v = np.array([x]).T+np.copysign(np.linalg.norm(x), x[0]) * np.array([e1]).T


        Ht = np.identity(x.shape[0])

        Ht -= (2 * (v @ v.T)/(v.T @ v))

        H[i:, i:] = Ht
        Q = Q @ H
        A = H @ A
        if inter:
            print(f'A{i}:\n', A.round(6)) 
    return Q, A

### d)

<i>À l’aide d’une matrice de dimension 4 × 3 de votre choix, testez votre fonction householder_qr et comparez les
résultats obtenus avec ceux obtenus à l’aide de la fonction numpy.linalg.qr. Les matrices sont-elles exactement
les mêmes ? Si non, est-ce un problème?</i>



In [352]:
a = np.random.randint(1,10,(4,3))
print(a)
Q, R = householder_qr(a)
print('Q:\n', Q.round(6))
print('R:\n', R.round(6))
r2 = np.linalg.qr(a, mode='complete')
print(r2)

[[7 3 4]
 [9 4 7]
 [5 2 7]
 [1 9 2]]
Q:
 [[-0.560449 -0.048025  0.493884 -0.663076]
 [-0.720577 -0.045024  0.097585  0.684996]
 [-0.40032  -0.051027 -0.863886 -0.301398]
 [-0.080064  0.996525 -0.016025 -0.01644 ]]
R:
 [[-12.489996  -6.08487  -10.248202]
 [  0.         8.542503   1.128595]
 [  0.         0.        -3.420619]
 [ -0.        -0.        -0.      ]]
QRResult(Q=array([[-0.56044854, -0.04802532,  0.49388393, -0.66307592],
       [-0.72057669, -0.04502373,  0.09758522,  0.68499578],
       [-0.40032038, -0.0510269 , -0.86388599, -0.30139814],
       [-0.08006408,  0.99652531, -0.01602452, -0.0164399 ]]), R=array([[-12.489996  ,  -6.08486984, -10.24820184],
       [  0.        ,   8.54250309,   1.12859493],
       [  0.        ,   0.        ,  -3.42061873],
       [  0.        ,   0.        ,   0.        ]]))


Les matrices sont les mêmes, mais la matrice Q pourrait avoir des signes différents selon la convention de signe utilisée qui menerait à la même matrice R.

### e)

<i>À l’aide de la matrice utilisée en d, illustrez comment la multiplication successive des matrices Q_i triangularise
progressivement la matrice A. Dans l’élan, assurez-vous que les matrices Q et R obtenues sont bien orthogonale et
triangulaire supérieure, respectivement.</i>


In [353]:
Q, R = householder_qr(a, inter=True)
I = Q @ Q.T
print(I)

A0:
 [[-12.489996  -6.08487  -10.248202]
 [ -0.        -0.195169   0.420531]
 [ -0.        -0.330649   3.34474 ]
 [  0.         8.53387    1.268948]]
A1:
 [[-12.489996  -6.08487  -10.248202]
 [  0.         8.542503   1.128595]
 [ -0.         0.         3.371534]
 [ -0.        -0.         0.5774  ]]
A2:
 [[-12.489996  -6.08487  -10.248202]
 [  0.         8.542503   1.128595]
 [  0.         0.        -3.420619]
 [ -0.        -0.        -0.      ]]
[[ 1.00000000e+00  7.09177005e-17  1.81596931e-16  9.99107517e-17]
 [ 7.09177005e-17  1.00000000e+00  2.73652423e-17 -6.79043042e-17]
 [ 1.81596931e-16  2.73652423e-17  1.00000000e+00 -7.59132184e-19]
 [ 9.99107517e-17 -6.79043042e-17 -7.59132184e-19  1.00000000e+00]]


On oberve qu'à chaque itération la colonne la plus à gauche non triangulaire le devient en devenant des zéros en dessous de la diagonale. En même temps, on observe que la dernière matrice A qui est égale à R est triangulaire supérieur que tous les valeurs en dessous de la diagonale sont zéros. Finalement, on vérifie l'orthogonalité de Q en multipliant Q par sa transpose. Le résultat obtenu est la matrice identité comme prévu, confirmant l'orthogonalité. À noter, que les zéros dans ce cas ne sont pas exactement zéro à cause d'erreur numérique.

In [354]:
I[I< 1e-15] = 0
print(I)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## TP2.2 Mesures imprécises dans un jeu de bataille navale

### a)

<i>Modifiez votre code de décomposition QR pour qu’il retourne la décomposition QR réduite de la matrice d’entrée
lorsque l’argument additionnel reduite=True lui est passé.</i>

In [355]:
def householder_qr(A, inter=False, reduite=False):
    m, n = A.shape
    Q = np.identity(m)
    for i in range(n):
        H = np.identity(m)
        x = A[i:, i]

        e1 = np.zeros(len(x)).T
        e1[0] = 1

        v = np.array([x]).T+np.copysign(np.linalg.norm(x), x[0]) * np.array([e1]).T


        Ht = np.identity(x.shape[0])

        Ht -= (2 * (v @ v.T)/(v.T @ v))

        H[i:, i:] = Ht
        Q = Q @ H
        A = H @ A
        if inter:
            print(f'A{i}:\n', A.round(6)) 
    if reduite:
        A = A[:n,:]
    return Q, A

### b)

<i>Utilisez votre code pour résoudre approximativement l’équation (2.2.4). Vous utiliserez les données fournies dans
le fichier bataille_navale_equipeXX.csv où vous remplacerez XX par votre numéro d’équipe dans la boîte de
dépôt sur MonPortail. </i>

In [394]:
data = readcsv(f'{path}\\{files_name[0]}', header=1)
x = data[:,0]
Y = data[:,1]
X = np.ones((len(x), 3))
X[:,1] = x
X[:,2] = x**2

Q,R = householder_qr(X, reduite=True)
print('Q:\n', Q.round(6))
print('R:\n', R.round(6))


right = Q @ Y
print(right)



Q:
 [[-0.353553 -0.540046  0.54002   0.007608  0.035958 -0.038041 -0.214222
  -0.492991]
 [-0.353553 -0.385767  0.077167  0.076219  0.238082  0.370226  0.47257
   0.545296]
 [-0.353553 -0.23151  -0.23139  -0.522749 -0.512008 -0.373887 -0.108636
   0.284342]
 [-0.353553 -0.077124 -0.385793  0.792707 -0.210879 -0.181231 -0.118408
  -0.022267]
 [-0.353553  0.07718  -0.385761 -0.219552  0.757668 -0.230617 -0.184454
  -0.10373 ]
 [-0.353553  0.231497 -0.231398 -0.171998 -0.213082  0.764175 -0.240227
  -0.226286]
 [-0.353553  0.385695  0.077007 -0.064731 -0.123217 -0.196897  0.714307
  -0.389787]
 [-0.353553  0.540075  0.540148  0.102495  0.027477 -0.113728 -0.320931
   0.405423]]
R:
 [[-2.82842700e+00 -6.75289279e+03 -1.61968471e+07]
 [ 0.00000000e+00 -4.58301915e+02 -2.18840078e+06]
 [-0.00000000e+00  0.00000000e+00  6.48168046e+04]]
[-538.11852814 1237.53158008 -718.36728091 -144.9052583  -169.97884223
 -106.33410749   71.18250068  337.40266231]


### c)

<i>Tracez les données (cercles noirs) et la solution estimée de la trajectoire(ligne pleine de la couleur de votre choix)
donnée par l’équation (2.2.3). </i>

### d)

<i>Obtenez la position d’impact du projectile (à y = 0) en résolvant l’équation quadratique (2.2.3) pour x à l’aide
d’une implémentation personnelle de la méthode de la bissection. Comparez votre solution avec celle obtenue en
résolvant cette même équation analytiquement. Considérant que votre embarcation se situe à la position (x, y) =
(0, 0), quelle est la distance horizontale vous séparant du point d’impact ?
 </i>